<a href="https://colab.research.google.com/github/Chayanonboo/dsb-batch10/blob/main/SQL_Restaurant_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**SQL Restaurant Database**

**Author**: Chayanon Boonthanom

**Language**: SQL

**Date**: 09 May 2025

เริ่มขั้นตอนดังนี้

ต้องมีการ Import pandas เพื่อแสดงผลลัพธ์เป็นตาราง

ส่วนของ SQL Script ทั้งหมด

จะแบ่งเป็นส่วน DDL/DML และส่วน Queries เพื่อความสะดวกในการจัดการ

In [2]:
import sqlite3
import pandas as pd

sql_setup_script = """
CREATE TABLE staff(
   staff_id int,
   staff_name Text,
   staff_position Text
);

INSERT INTO staff values
   (1, 'Non','Manager'),
   (2, 'Nan','Officer'),
   (3, 'Mind','Officer');

CREATE TABLE menu(
     menu_id INT,
     name_food TEXT,
     price REAL,
     category TEXT
);

INSERT INTO menu values
 (1,'Banana',10.00,'Fruit'),
 (2,'Rice with pork',50.00,'Thai Food'),
 (3,'Pizza',200.00,'Italian Food'),
 (4,'Sushi',120.00,'Japanese Food');

CREATE TABLE transaction_cooking(
     transaction_id INT,
     order_date DATETIME,
     customer_id INT,
     staff_id INT,
     total_amount REAL,
     category TEXT -- เพิ่ม category ใน transaction_cooking ให้สอดคล้องกับ JOIN
);

INSERT INTO transaction_cooking values
   (30,'2024-06-01',1,1,320.00,'Fruit'),          -- Banana (สมมติว่าลูกค้าซื้อ Banana หลายชิ้น)
   (31,'2024-06-02',2,2,600.00,'Thai Food'),       -- Rice with pork
   (32,'2024-06-03',3,3,610.00,'Italian Food'),    -- Pizza
   (33,'2024-06-03',4,1,240.00,'Italian Food'),    -- Pizza (อีกออเดอร์)
   (34,'2024-06-04',5,2,360.00,'Japanese Food'),   -- Sushi
   (35,'2024-06-04',6,3,380.00,'Japanese Food'),   -- Sushi
   (36,'2024-06-05',7,1,800.00,'Japanese Food'),   -- Sushi
   (37,'2024-06-05',8,2,1000.00,'Japanese Food'); -- Sushi

CREATE TABLE customers (
    customer_id int PRIMARY KEY,
    customer_name TEXT,
    gender TEXT
);

INSERT INTO customers values
     (1,'James','M'),
     (2,'Mary','F'),
     (3,'Onn','F'),
     (4,'Kaning','F'),
     (5,'Boss','M'),
     (6,'Son','F'),
     (7,'palm','F'),
     (8,'fern','M');
"""


สร้างการเชื่อมต่อ (Connection) และ Cursor

สร้างฐานข้อมูลในหน่วยความจำ


In [3]:
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()


 ลอง Execute SQL Setup Script (CREATE TABLEs และ INSERT INTO data) ว่าสามารถใช้ได้หรือเปล่า

 conn.commit() = ยืนยันการเปลี่ยนแปลงข้อมูล

In [4]:
try:
    cursor.executescript(sql_setup_script)
    conn.commit() # ยืนยันการเปลี่ยนแปลงข้อมูล
    print("Tables created and data inserted successfully.")
except sqlite3.Error as e:
    print(f"An error occurred: {e}")



Tables created and data inserted successfully.


รัน SELECT Queries ทีละ Query และแสดงผลด้วย Pandas

**Query 1: Where clause staff table**

In [5]:
print("\n--- Query 1: Where clause staff table ---")
query1 = """
SELECT
    staff_id,
    staff_name
FROM staff
WHERE staff_name = 'Non';
"""
df1 = pd.read_sql_query(query1, conn)
print(df1)




--- Query 1: Where clause staff table ---
   staff_id staff_name
0         1        Non


**Query 2: Where clause menu table**

In [6]:
print("\n--- Query 2: Where clause menu table (แสดงทั้งหมด) ---")
# หมายเหตุ: ใน SQL เดิมของคุณไม่ได้ใส่ WHERE clause สำหรับ menu แต่เป็นการ select *
# ถ้าต้องการ WHERE clause ให้เพิ่มเข้าไป
query2 = """
SELECT
    name_food,
    price
FROM MENU;
"""
df2 = pd.read_sql_query(query2, conn)
print(df2)




--- Query 2: Where clause menu table (แสดงทั้งหมด) ---
        name_food  price
0          Banana   10.0
1  Rice with pork   50.0
2           Pizza  200.0
3           Sushi  120.0


**Query 3: Aggregate function for transaction_cooking**

In [7]:
print("\n--- Query 3: Aggregate function for transaction_cooking ---")
query3 = """
SELECT
    order_date,
    Count(*) AS num_transactions,
    AVG(total_amount) AS avg_amount,
    SUM(total_amount) AS sum_amount,
    MAX(total_amount) AS max_amount,
    MIN(total_amount) AS min_amount
FROM transaction_cooking
GROUP BY order_date;
"""
df3 = pd.read_sql_query(query3, conn)
print(df3)




--- Query 3: Aggregate function for transaction_cooking ---
   order_date  num_transactions  avg_amount  sum_amount  max_amount  \
0  2024-06-01                 1       320.0       320.0       320.0   
1  2024-06-02                 1       600.0       600.0       600.0   
2  2024-06-03                 2       425.0       850.0       610.0   
3  2024-06-04                 2       370.0       740.0       380.0   
4  2024-06-05                 2       900.0      1800.0      1000.0   

   min_amount  
0       320.0  
1       600.0  
2       240.0  
3       360.0  
4       800.0  


**Query 4: Subquery with Staff table (staff_id = 1)**

ใน SQL เดิม WHERE staff_id = '1' ซึ่ง '1' เป็น TEXT

ถ้า staff_id เป็น INT ควรจะเป็น WHERE staff_id = 1

In [8]:
print("\n--- Query 4: Subquery with Staff table (staff_id = 1) ---")
query4 = """
SELECT
    staff_name,
    staff_position
FROM (
    SELECT *
    FROM staff
    WHERE staff_id = 1  -- แก้ไขเป็น 1 (INT)
);
"""
df4 = pd.read_sql_query(query4, conn)
print(df4)





--- Query 4: Subquery with Staff table (staff_id = 1) ---
  staff_name staff_position
0        Non        Manager


**Query 5: WITH clause (CTE) joining multiple tables**

In [9]:
print("\n--- Query 5: WITH clause (CTE) joining multiple tables ---")
query5 = """
WITH customer_detail AS (
    SELECT
        sta.staff_name AS staff_name,
        sta.staff_position AS staff_position,
        cus.customer_name AS customer_name,
        cus.gender AS gender,
        men.name_food AS name_food,
        men.price AS price_per_unit, -- เปลี่ยนชื่อ price เพื่อไม่ให้สับสนกับ total_amount
        tra.total_amount AS amount_receive,
        tra.order_date,
        tra.transaction_id
    FROM staff AS sta
    JOIN transaction_cooking AS tra ON sta.staff_id = tra.staff_id
    JOIN customers AS cus ON tra.customer_id = cus.customer_id
    JOIN menu AS men ON tra.category = men.category -- JOIN menu ด้วย category จาก transaction_cooking
)
SELECT
    transaction_id,
    order_date,
    staff_name,
    staff_position,
    customer_name,
    gender,
    name_food,
    price_per_unit,
    amount_receive
FROM customer_detail
ORDER BY order_date, transaction_id;
"""
df5 = pd.read_sql_query(query5, conn)
print(df5)


--- Query 5: WITH clause (CTE) joining multiple tables ---
   transaction_id  order_date staff_name staff_position customer_name gender  \
0              30  2024-06-01        Non        Manager         James      M   
1              31  2024-06-02        Nan        Officer          Mary      F   
2              32  2024-06-03       Mind        Officer           Onn      F   
3              33  2024-06-03        Non        Manager        Kaning      F   
4              34  2024-06-04        Nan        Officer          Boss      M   
5              35  2024-06-04       Mind        Officer           Son      F   
6              36  2024-06-05        Non        Manager          palm      F   
7              37  2024-06-05        Nan        Officer          fern      M   

        name_food  price_per_unit  amount_receive  
0          Banana            10.0           320.0  
1  Rice with pork            50.0           600.0  
2           Pizza           200.0           610.0  
3          

ปิดการเชื่อมต่อ

In [10]:
conn.close()
print("\nDatabase connection closed.")


Database connection closed.
